# Classificatore BERT

In [ ]:
# Lettura dei dataset di training, test e validation
train_data= pd.read_csv('/content/drive/My Drive/TMAG_Classification_BERT/CItA4Finetuning/CItA_150/Train_CItA_Finetuning.csv', delimiter=",")
test_data = pd.read_csv('/content/drive/My Drive/TMAG_Classification_BERT/CItA4Finetuning/CItA_150/Test_CItA_Finetuning.csv', delimiter=",")
val_data = pd.read_csv('/content/drive/My Drive/TMAG_Classification_BERT/CItA4Finetuning/CItA_150/Val_CItA_Finetuning.csv', delimiter=",")

# Selezione delle feature d'interesse per ognuno (saggi e label)
train_data=train_data[["Formatted_Text","Order_Label"]].reset_index(drop=True)
test_data=test_data[["Formatted_Text","Order_Label"]].reset_index(drop=True)
val_data=val_data[["Formatted_Text","Order_Label"]].reset_index(drop=True)

# Conversione dati in specifico tipo datasets
train = datasets.Dataset.from_pandas(pd.DataFrame(data=train_data))
test = datasets.Dataset.from_pandas(pd.DataFrame(data=test_data))
dev = datasets.Dataset.from_pandas(pd.DataFrame(data=val_data))

# Creazione variabile 'model' con il modello da utilizzare
model_name= "dbmdz/bert-base-italian-xxl-uncased"
# Caricamento del modello
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
# Caricamento del tokenizzatore
tokenizer = AutoTokenizer.from_pretrained(model_name)
                                                
# Funzione che applica la tokenizzazione ai dati di training, dev e test
def tokenize(batch):

    # Applica tokenizzazione ai testi
    # Limite imposto a 512 subwords, di conseguenza se la sequenza è più lunga viene troncata, se invece non arriva a 512 vengono inseriti degli zeri
    tokens = tokenizer(batch['Formatted_Text'], padding=True, truncation=True, max_length=512)
    # Assegna le label 
    tokens['label'] = batch["Order_Label"]
    return tokens

# Tokenizzazione dei set usando tokenizer del modello
train= train.map(tokenize, batched=True)
dev = dev.map(tokenize, batched=True)
test = test.map(tokenize, batched=True)

# Format dei set per il modello
train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
dev.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])    
                                                
# Definizione degli argomenti per l'addestramento
num_epochs = 2
training_args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    logging_strategy="epoch",
    save_strategy = "epoch",
    num_train_epochs=num_epochs,
    logging_steps=15,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.005,
    load_best_model_at_end=True
)
                                                
# La funzione prende in input le predictions del modello
def compute_metrics(eval_pred):
    f1_metric = evaluate.load("f1")
    predictions, labels = eval_pred
    # Applica argmax alle predictions
    predictions = np.argmax(predictions, axis=1)
    # Calcola e restituisce l'F-Score fra le predictions e le true labels
    return f1_metric.compute(predictions=predictions, references=labels, average="weighted")

# Definizione del trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=train,
    eval_dataset=dev,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Adddestramento
trainer.train()
# Salvataggio del modello al percorso specificato 
trainer.save_model("/content/drive/My Drive/TMAG_Classification_BERT/CItA4Finetuning/Model_ft/lenght150/FTmode_2ep__set3")
                                                
# Estrazione cronologia dei log dall'oggetto trainer
log_history = trainer.state.log_history

# Creazione dataframe per memorizzare le loss di training e validation
df = pd.DataFrame(columns=["Epoch", "Loss", "Training/Validation"])

# Itera sulla cronologia dei log
for log_data in log_history:
    
    epoch = int(log_data["epoch"])
    # Se è presente una chiave "loss" nel log
    if "loss" in log_data.keys():
    loss = log_data["loss"]
    # Memorizza la loss di training nel dataframe
    df = df.append({"Epoch": epoch, "Loss": loss, "Training/Validation": "Training"}, ignore_index=True)

    # Se è presente una chiave "eval_loss" nel log
    if "eval_loss" in log_data.keys():
    loss = log_data["eval_loss"]
    # Memorizza la loss di validation nel dataframe
    df = df.append({"Epoch": epoch, "Loss": loss, "Training/Validation": "Validation"}, ignore_index=True)

# Creazione lineplot della loss
sns.lineplot(data=df, x="Epoch", y="Loss", hue="Training/Validation")
                                                
# Ricava le prediction del modello
output_predictions = trainer.predict(test)

# Ottienimento delle label reali dal test set
y_test = test["label"].tolist()
# Ottienimento delle label predette dal modello
y_pred = np.argmax(output_predictions.predictions, axis=1)

# Calcola il classification report
report= classification_report(y_test, y_pred)
# Crea una rappresentazione grafica della matrice di confusione
cm = ConfusionMatrixDisplay.from_predictions(y_test, y_pred, xticks_rotation='vertical', cmap='Blues')

print("Classification Report su test set:")
print(report, "\n")
print("Confusion Matrix su test set:")
print(cm)